# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [236]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [237]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [238]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = []
    target_id_text = []
    
    for line in source_text.split('\n'):
        source_id_text.append([source_vocab_to_int[word] for word in line.split()])
    
    for line in target_text.split('\n'):
        target_id_text.append([target_vocab_to_int[word] for word in line.split()] + [target_vocab_to_int['<EOS>']])
        
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [239]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [240]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [241]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1


C:\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\ipykernel\__main__.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [242]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_  = tf.placeholder(tf.int32, shape=(None,None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None,None))
    
    learningrate = tf.placeholder(tf.float32, shape=[])
    keep_prob    = tf.placeholder(tf.float32, shape=[], name='keep_prob')
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_length    = tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(tf.int32, [None], name='source_sequence_length')
    
    return (input_,targets,
            learningrate,keep_prob,
            target_sequence_length,max_target_length,
            source_sequence_length)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [243]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    #begin: An int32 or int64 Tensor.
    #end: An int32 or int64 Tensor.
    #strides: An int32 or int64 Tensor.
    
    strid = tf.strided_slice(target_data, [0,0],[batch_size,-1],[1,1] )   
    target_data = tf.concat( [tf.fill([batch_size,1],target_vocab_to_int['<GO>']),strid] ,1)
    
    return target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [244]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """   
    def lstm_cell(rnn_size):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm = tf.contrib.rnn.DropoutWrapper(lstm, keep_prob)
        return lstm
    
    embed_seq = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
        
    RNN_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])

    output,state = tf.nn.dynamic_rnn(RNN_cell, embed_seq, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return RNN_cell,state
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [245]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length)
        
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,training_helper,
                                                    encoder_state,output_layer) 
        
    dynamic_decoder = tf.contrib.seq2seq.dynamic_decode(basic_decoder,
                                                        maximum_iterations=max_summary_length)[0]
    return dynamic_decoder





"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [246]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function    
    greedy_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens=[start_of_sequence_id]*batch_size ,
                                                                end_token=end_of_sequence_id)

    basic_decoder  = tf.contrib.seq2seq.BasicDecoder(dec_cell,greedy_helper,
                                                        encoder_state,output_layer)
        
    dynamic_decode = tf.contrib.seq2seq.dynamic_decode(basic_decoder,
                                                            maximum_iterations=max_target_sequence_length)[0]
    
    return dynamic_decode



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [247]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """    # TODO: Implement Function
    def ltsm_cell(rnn_size, keep_prob):
        cell = tf.contrib.rnn.LSTMCell(rnn_size)
        cell = tf.contrib.rnn.DropoutWrapper(cell, keep_prob)
        return cell
    
    dec_rnd         = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_rnd, dec_input)
    
    dec_cell = tf.contrib.rnn.MultiRNNCell([ltsm_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    output_layer = Dense(target_vocab_size)
    
    with tf.variable_scope("decode"):
        training_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                                 target_sequence_length, max_target_sequence_length, 
                                 output_layer, keep_prob)
    
    start_of_sequence_id = target_vocab_to_int['<GO>' ]
    end_of_sequence_id   = target_vocab_to_int['<EOS>']
    
    with tf.variable_scope("decode"):
        inference_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_rnd, start_of_sequence_id,
                                 end_of_sequence_id, max_target_sequence_length,
                                 target_vocab_size, output_layer, batch_size, keep_prob)
        
    return training_decoder_output, inference_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [248]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size,enc_embedding_size)[1] #tpl
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    basic_training_decoder_output, basic_decoder_output = decoding_layer(dec_input, enc_state,
                   target_sequence_length, max_target_sentence_length,rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,batch_size, keep_prob, dec_embedding_size)
    
    return basic_training_decoder_output, basic_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [249]:
# Number of Epochs
epochs = 5
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 64
# Number of Layers
num_layers = 3
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.5
display_step = 1

### Build the Graph
Build the graph using the neural network you implemented.

In [250]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [251]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [253]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2329, Validation Accuracy: 0.3096, Loss: 5.6921
Epoch   0 Batch    2/269 - Train Accuracy: 0.2655, Validation Accuracy: 0.3096, Loss: 5.3640
Epoch   0 Batch    3/269 - Train Accuracy: 0.2444, Validation Accuracy: 0.3096, Loss: 5.0551
Epoch   0 Batch    4/269 - Train Accuracy: 0.2437, Validation Accuracy: 0.3220, Loss: 4.7415
Epoch   0 Batch    5/269 - Train Accuracy: 0.2442, Validation Accuracy: 0.3220, Loss: 4.4238
Epoch   0 Batch    6/269 - Train Accuracy: 0.2903, Validation Accuracy: 0.3220, Loss: 4.0113
Epoch   0 Batch    7/269 - Train Accuracy: 0.3068, Validation Accuracy: 0.3399, Loss: 3.7841
Epoch   0 Batch    8/269 - Train Accuracy: 0.2764, Validation Accuracy: 0.3411, Loss: 3.7431
Epoch   0 Batch    9/269 - Train Accuracy: 0.3029, Validation Accuracy: 0.3428, Loss: 3.5572
Epoch   0 Batch   10/269 - Train Accuracy: 0.2709, Validation Accuracy: 0.3427, Loss: 3.5944
Epoch   0 Batch   11/269 - Train Accuracy: 0.3077, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4092, Validation Accuracy: 0.4609, Loss: 2.5198
Epoch   0 Batch   91/269 - Train Accuracy: 0.4682, Validation Accuracy: 0.4893, Loss: 2.3673
Epoch   0 Batch   92/269 - Train Accuracy: 0.4651, Validation Accuracy: 0.4883, Loss: 2.3663
Epoch   0 Batch   93/269 - Train Accuracy: 0.4585, Validation Accuracy: 0.4524, Loss: 2.2873
Epoch   0 Batch   94/269 - Train Accuracy: 0.4515, Validation Accuracy: 0.4664, Loss: 2.3702
Epoch   0 Batch   95/269 - Train Accuracy: 0.4646, Validation Accuracy: 0.4880, Loss: 2.3541
Epoch   0 Batch   96/269 - Train Accuracy: 0.4547, Validation Accuracy: 0.4839, Loss: 2.3613
Epoch   0 Batch   97/269 - Train Accuracy: 0.4442, Validation Accuracy: 0.4593, Loss: 2.3339
Epoch   0 Batch   98/269 - Train Accuracy: 0.4796, Validation Accuracy: 0.4906, Loss: 2.3166
Epoch   0 Batch   99/269 - Train Accuracy: 0.4470, Validation Accuracy: 0.4994, Loss: 2.4527
Epoch   0 Batch  100/269 - Train Accuracy: 0.4805, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.5328, Validation Accuracy: 0.5486, Loss: 1.8035
Epoch   0 Batch  180/269 - Train Accuracy: 0.5246, Validation Accuracy: 0.5478, Loss: 1.7852
Epoch   0 Batch  181/269 - Train Accuracy: 0.5296, Validation Accuracy: 0.5510, Loss: 1.7737
Epoch   0 Batch  182/269 - Train Accuracy: 0.5188, Validation Accuracy: 0.5392, Loss: 1.7996
Epoch   0 Batch  183/269 - Train Accuracy: 0.5855, Validation Accuracy: 0.5415, Loss: 1.5736
Epoch   0 Batch  184/269 - Train Accuracy: 0.5107, Validation Accuracy: 0.5498, Loss: 1.8419
Epoch   0 Batch  185/269 - Train Accuracy: 0.5376, Validation Accuracy: 0.5484, Loss: 1.7218
Epoch   0 Batch  186/269 - Train Accuracy: 0.4954, Validation Accuracy: 0.5427, Loss: 1.8344
Epoch   0 Batch  187/269 - Train Accuracy: 0.5264, Validation Accuracy: 0.5409, Loss: 1.7068
Epoch   0 Batch  188/269 - Train Accuracy: 0.5379, Validation Accuracy: 0.5421, Loss: 1.6844
Epoch   0 Batch  189/269 - Train Accuracy: 0.5370, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.4769, Validation Accuracy: 0.5134, Loss: 1.2136
Epoch   1 Batch    2/269 - Train Accuracy: 0.4711, Validation Accuracy: 0.4897, Loss: 1.1956
Epoch   1 Batch    3/269 - Train Accuracy: 0.4612, Validation Accuracy: 0.4886, Loss: 1.2112
Epoch   1 Batch    4/269 - Train Accuracy: 0.4699, Validation Accuracy: 0.5134, Loss: 1.2052
Epoch   1 Batch    5/269 - Train Accuracy: 0.4649, Validation Accuracy: 0.5233, Loss: 1.2155
Epoch   1 Batch    6/269 - Train Accuracy: 0.5170, Validation Accuracy: 0.5217, Loss: 1.1087
Epoch   1 Batch    7/269 - Train Accuracy: 0.4884, Validation Accuracy: 0.4979, Loss: 1.1385
Epoch   1 Batch    8/269 - Train Accuracy: 0.4629, Validation Accuracy: 0.4950, Loss: 1.1926
Epoch   1 Batch    9/269 - Train Accuracy: 0.4788, Validation Accuracy: 0.4924, Loss: 1.1490
Epoch   1 Batch   10/269 - Train Accuracy: 0.4813, Validation Accuracy: 0.5067, Loss: 1.1638
Epoch   1 Batch   11/269 - Train Accuracy: 0.4791, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.4971, Validation Accuracy: 0.5444, Loss: 0.9725
Epoch   1 Batch   91/269 - Train Accuracy: 0.5178, Validation Accuracy: 0.5403, Loss: 0.8989
Epoch   1 Batch   92/269 - Train Accuracy: 0.5361, Validation Accuracy: 0.5489, Loss: 0.9011
Epoch   1 Batch   93/269 - Train Accuracy: 0.5489, Validation Accuracy: 0.5518, Loss: 0.8773
Epoch   1 Batch   94/269 - Train Accuracy: 0.5282, Validation Accuracy: 0.5474, Loss: 0.9218
Epoch   1 Batch   95/269 - Train Accuracy: 0.5311, Validation Accuracy: 0.5469, Loss: 0.9149
Epoch   1 Batch   96/269 - Train Accuracy: 0.5307, Validation Accuracy: 0.5448, Loss: 0.9054
Epoch   1 Batch   97/269 - Train Accuracy: 0.5266, Validation Accuracy: 0.5456, Loss: 0.9038
Epoch   1 Batch   98/269 - Train Accuracy: 0.5568, Validation Accuracy: 0.5518, Loss: 0.8976
Epoch   1 Batch   99/269 - Train Accuracy: 0.5187, Validation Accuracy: 0.5515, Loss: 0.9467
Epoch   1 Batch  100/269 - Train Accuracy: 0.5559, Validation Accuracy

...

Epoch   3 Batch  179/269 - Train Accuracy: 0.6948, Validation Accuracy: 0.7071, Loss: 0.4466
Epoch   3 Batch  180/269 - Train Accuracy: 0.7104, Validation Accuracy: 0.7026, Loss: 0.4410
Epoch   3 Batch  181/269 - Train Accuracy: 0.7134, Validation Accuracy: 0.7108, Loss: 0.4443
Epoch   3 Batch  182/269 - Train Accuracy: 0.7240, Validation Accuracy: 0.7100, Loss: 0.4325
Epoch   3 Batch  183/269 - Train Accuracy: 0.7522, Validation Accuracy: 0.7120, Loss: 0.3851
Epoch   3 Batch  184/269 - Train Accuracy: 0.6969, Validation Accuracy: 0.7124, Loss: 0.4525
Epoch   3 Batch  185/269 - Train Accuracy: 0.7046, Validation Accuracy: 0.7058, Loss: 0.4336
Epoch   3 Batch  186/269 - Train Accuracy: 0.6811, Validation Accuracy: 0.7053, Loss: 0.4527
Epoch   3 Batch  187/269 - Train Accuracy: 0.7091, Validation Accuracy: 0.7040, Loss: 0.4221
Epoch   3 Batch  188/269 - Train Accuracy: 0.7104, Validation Accuracy: 0.7103, Loss: 0.4256
Epoch   3 Batch  189/269 - Train Accuracy: 0.7168, Validation Accuracy

Epoch   4 Batch    1/269 - Train Accuracy: 0.7144, Validation Accuracy: 0.7256, Loss: 0.4157
Epoch   4 Batch    2/269 - Train Accuracy: 0.7248, Validation Accuracy: 0.7235, Loss: 0.4071
Epoch   4 Batch    3/269 - Train Accuracy: 0.7354, Validation Accuracy: 0.7263, Loss: 0.4185
Epoch   4 Batch    4/269 - Train Accuracy: 0.7146, Validation Accuracy: 0.7275, Loss: 0.4175
Epoch   4 Batch    5/269 - Train Accuracy: 0.7195, Validation Accuracy: 0.7264, Loss: 0.4180
Epoch   4 Batch    6/269 - Train Accuracy: 0.7393, Validation Accuracy: 0.7243, Loss: 0.3941
Epoch   4 Batch    7/269 - Train Accuracy: 0.7262, Validation Accuracy: 0.7243, Loss: 0.3999
Epoch   4 Batch    8/269 - Train Accuracy: 0.6919, Validation Accuracy: 0.7210, Loss: 0.4199
Epoch   4 Batch    9/269 - Train Accuracy: 0.7111, Validation Accuracy: 0.7201, Loss: 0.4145
Epoch   4 Batch   10/269 - Train Accuracy: 0.7278, Validation Accuracy: 0.7321, Loss: 0.4151
Epoch   4 Batch   11/269 - Train Accuracy: 0.7267, Validation Accuracy

Epoch   4 Batch   90/269 - Train Accuracy: 0.7265, Validation Accuracy: 0.7454, Loss: 0.3976
Epoch   4 Batch   91/269 - Train Accuracy: 0.7424, Validation Accuracy: 0.7499, Loss: 0.3692
Epoch   4 Batch   92/269 - Train Accuracy: 0.7605, Validation Accuracy: 0.7514, Loss: 0.3716
Epoch   4 Batch   93/269 - Train Accuracy: 0.7623, Validation Accuracy: 0.7528, Loss: 0.3579
Epoch   4 Batch   94/269 - Train Accuracy: 0.7256, Validation Accuracy: 0.7550, Loss: 0.3854
Epoch   4 Batch   95/269 - Train Accuracy: 0.7433, Validation Accuracy: 0.7512, Loss: 0.3745
Epoch   4 Batch   96/269 - Train Accuracy: 0.7387, Validation Accuracy: 0.7510, Loss: 0.3846
Epoch   4 Batch   97/269 - Train Accuracy: 0.7487, Validation Accuracy: 0.7532, Loss: 0.3726
Epoch   4 Batch   98/269 - Train Accuracy: 0.7391, Validation Accuracy: 0.7509, Loss: 0.3758
Epoch   4 Batch   99/269 - Train Accuracy: 0.7413, Validation Accuracy: 0.7576, Loss: 0.3887
Epoch   4 Batch  100/269 - Train Accuracy: 0.7883, Validation Accuracy

Epoch   4 Batch  179/269 - Train Accuracy: 0.7503, Validation Accuracy: 0.7613, Loss: 0.3550
Epoch   4 Batch  180/269 - Train Accuracy: 0.7768, Validation Accuracy: 0.7628, Loss: 0.3516
Epoch   4 Batch  181/269 - Train Accuracy: 0.7478, Validation Accuracy: 0.7532, Loss: 0.3558
Epoch   4 Batch  182/269 - Train Accuracy: 0.7626, Validation Accuracy: 0.7495, Loss: 0.3517
Epoch   4 Batch  183/269 - Train Accuracy: 0.7734, Validation Accuracy: 0.7531, Loss: 0.3122
Epoch   4 Batch  184/269 - Train Accuracy: 0.7349, Validation Accuracy: 0.7542, Loss: 0.3629
Epoch   4 Batch  185/269 - Train Accuracy: 0.7540, Validation Accuracy: 0.7551, Loss: 0.3478
Epoch   4 Batch  186/269 - Train Accuracy: 0.7306, Validation Accuracy: 0.7519, Loss: 0.3640
Epoch   4 Batch  187/269 - Train Accuracy: 0.7553, Validation Accuracy: 0.7573, Loss: 0.3435
Epoch   4 Batch  188/269 - Train Accuracy: 0.7666, Validation Accuracy: 0.7578, Loss: 0.3455
Epoch   4 Batch  189/269 - Train Accuracy: 0.7520, Validation Accuracy

Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [254]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [255]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [256]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    word_ids =  [vocab_to_int[word] if word in vocab_to_int.keys() else vocab_to_int['<UNK>'] for word in sentence.split()]
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [257]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [18, 156, 220, 148, 21, 213, 88]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [125, 86, 96, 109, 64, 106, 28, 1]
  French Words: il a vu une voiture verte . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.